# Wayfarer: Intelligent Entity Eviction 🐬

##### 💡 **Research Areas:** Rapid Prototyping, Generative AI, Data Eviction Systems, System Design.

#### I had an interesting idea — how about a system which stores entities (objects) and processes incoming events. Based on the rules and semantics of the incoming event, certain mutations / operations are done on the data. For now, I decided to keep it simple — incoming events have rules or fields that decide the nature of data in the system, based on the events, the entities that don't fulfil the said conditions are evicted in real-time!

<div style="display:flex; align-items:center; padding: 50px;">
<p style="margin-right:10px;">
    <img height="300px" style="width:auto;" width="200px" src="https://avatars.githubusercontent.com/u/192148546?s=400&u=95d76fbb02e6c09671d87c9155f17ca1e4ef8f21&v=4"> 
</p>
</div>

## Presenting Wayfarer. Let's see how that works!


In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv()

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

In [7]:
from typing import Dict, Any, List
import json
import traceback
from openai import OpenAI
from uuid import uuid4
from copy import deepcopy
import os

DEFAULT_OPENAI_MODEL = "gpt-4o-mini"
DEFAULT_SIMILARITY_THRESHOLD = 0.9


class Wayfarer:
    """Wayfarer: An AI-powered, intelligent system to retrieve entities based on real-time events using natural language queries.

    Background: One of the advantages of Generative AI is the ability to program computer systems using natural language.
    During our work at The Hackers Playbook, we exercised our creativity and thought of a unique use-case for real-time, event-driven systems.
    Consider systems where events keep entering the system, and you want to retrieve entities that match some 'rule' or 'criteria' in the incoming event(s).
    That's the problem we're solving with Wayfarer.

    System Design: Wayfarer's Design is simple. It uses LLM's to detect if an entity in the entites set matches the incoming event(s).
    If there is match with any of the provided events, the entity is evicted and returned to the caller.
    We use an 'event log' (dictionary) to keep track of events in the system and a list to store the entities.
    While it might seem inefficient to use a list because retrieval becomes O(n), we could consider optimizing and using a more efficient data structure in future.

    Disclaimer: Wayfarer is currently experimental and in alpha stage so use at your own risk.
    TODO: Add a event based task(s) trigger system to trigger tasks based on events.
    """

    def __init__(self, model=DEFAULT_OPENAI_MODEL, debug=True, verbose=True):
        """Initializes Wayfarer."""
        self.entities = []
        self.event_log = {}
        self.model = model
        self.llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
        self.debug = debug
        self.verbose = verbose
        print(f"🐬 Wayfarer initialized with AI model: {self.model} 🌊")
        print(
            "👉🏽 Debug mode is enabled." if self.debug else "Debug mode is disabled."
        )
        print(
            "👉🏽 Verbose mode is enabled."
            if self.verbose
            else "Verbose mode is disabled."
        )

    def entity_exists(
        self, entity: Dict[str, Any], similarity_threshold=DEFAULT_SIMILARITY_THRESHOLD
    ) -> bool:
        """Checks if the entity exists in the entities set."""
        if self.verbose:
            print("Checking if the entity exists: ", entity)

        if len(self.entities) == 0:
            return False

        for cur_entity in self.entities:
            cur_entity_json = json.dumps(cur_entity)
            entity_json = json.dumps(entity)

            try:
                prompt = f"""
                    Given the JSON of 2 entities, provide a similarity score between them.

                    The similarity score should take into account the 'semantic' similarity between the entities.
                    It should be high if the entities are similar and low if they are different.
                    You can ignore the 'id' field while comparing the entities.

                    Reflection: Review the similarity score and evaluate if your similarity score accurately reflects the similarity between the entities.
                    If it doesn't, consider adjusting the similarity threshold.
                    Repeat this a few times till you have a final similarity score. 
                    Return the final similarity score. 
                    Respond only with a number between 0 and 1. STRICTLY NO TEXT ONLY NUMBER BETWEEN 0 and 1.
                    Entity 1: '''{cur_entity_json}'''
                    Entity 2: '''{entity_json}'''
                """
                system = (
                    "You are an AI assistant that helps me compare entities (JSONs)."
                )
                response = self.llm.chat.completions.create(
                    messages=[
                        {"role": "system", "content": system},
                        {"role": "user", "content": prompt},
                    ],
                    model=self.model,
                )
                result = response.choices[0].message.content
                similarity_score = float(result)
                if self.verbose:
                    print(f"Similarity Score: {similarity_score}")
                if similarity_score >= similarity_threshold:
                    return True
            except Exception as e:
                error_message = f"An error occurred while checking if the entity exists, assuming it doesn't exist."
                print(error_message)
                if self.debug:
                    print(f"Error: {e}")
                    traceback.print_exc()
                continue
        return False

    def entities_exist(
        self,
        entities: List[Dict[str, Any]],
        similarity_threshold=DEFAULT_SIMILARITY_THRESHOLD,
    ) -> List[bool]:
        """Checks if the entities (multiple) exist in the entities set."""
        results = []

        if self.verbose:
            print("Checking if the entities exist: ", entities)

        if len(self.entities) == 0:
            return [False for _ in entities]

        for entity in entities:
            results.append(self.entity_exists(entity, similarity_threshold))
        return results

    def add_event(self, event: Dict[str, Any]):
        """Adds an event to the event log."""
        if self.verbose:
            print("Adding event to the event log: ", event)
        event_id = event.get("id", str(uuid4()))

        if event_id in self.event_log:
            if self.debug:
                print(f"Event with ID {event_id} already exists in the event log.")
            return event_id
        self.event_log[event_id] = event
        return event_id

    def remove_event(self, event_id: str) -> bool:
        """Removes an event from the event log."""
        if self.verbose:
            print("Removing event from the event log: ", event_id)
        if event_id in self.event_log:
            del self.event_log[event_id]
            return True
        return False

    def add_entity(
        self, entity: Dict[str, Any], similarity_threshold=DEFAULT_SIMILARITY_THRESHOLD
    ) -> Dict[str, Any]:
        """Adds an entity to the entities set."""
        if self.verbose:
            print("Adding entity to the entities set: ", entity)

        if not entity:
            if self.verbose:
                print("Entity is empty. Skipping...")
            return None

        entity_id = entity.get("id", str(uuid4()))
        entity["id"] = entity_id

        if self.entity_exists(entity, similarity_threshold=similarity_threshold):
            if self.debug:
                print(f"Similar entity {entity} already exists in the entities set.")
            return entity

        self.entities.append(entity)
        return deepcopy(entity)

    def remove_entity(self, entity_id: str) -> bool:
        """Removes an entity from the entities set."""
        if self.verbose:
            print("Removing entity from the entities set: ", entity_id)
        for entity in self.entities:
            if entity.get("id") == entity_id:
                self.entities.remove(entity)
                return True
        return False

    def get_entities(self) -> List[Dict[str, Any]]:
        """Returns the entities set."""
        return deepcopy(self.entities)  # performance drop but ensures immutability

    def get_event_log(self) -> Dict[str, Any]:
        """Returns the event log."""
        return deepcopy(self.event_log)

    def set_verbose(self, verbose: bool):
        """Sets the verbose mode."""
        self.verbose = verbose

    def set_debug(self, debug: bool):
        """Sets the debug mode."""
        self.debug = debug

    def set_model(self, model: str):
        """Sets the model."""
        self.model = model

    def reset(self):
        """Resets the Wayfarer instance."""
        self.entities = []
        self.event_log = {}

    def get_evictions(self) -> List[Dict[str, Any]]:
        """Returns the entities that are evicted based on the events."""
        evictions = []
        for event_id, event in self.event_log.items():
            for entity in self.entities:
                try:
                    event_json = json.dumps(event)
                    entity_json = json.dumps(entity)
                    if self.verbose:
                        print(
                            f"Processing event[{event_id}] {event_json} and {entity_json} for eviction."
                        )
                    entity_json = json.dumps(entity)
                    system = "You are an AI assistant that helps me determine if an entity should be evicted based on an event."
                    prompt = f"""
                    Given an entity and an event. 
                    - Check if the event contains any 'rule' or 'condition' or 'field' that implies the entity should be evicted.
                    - Check if the entity contains any 'rule' or 'condition' or 'field' that implies the event should be evicted.
                    - For instance John Doe, Age 12 will be removed if 'event metadata' contains 'Remove if Age > 10'.
                    - Respond with 'Yes' or 'No'. Yes means the entity should be evicted. No means it should not be evicted.
                    Entity: {entity_json}
                    Event: {event_json}
                    """
                    response = self.llm.chat.completions.create(
                        messages=[
                            {"role": "system", "content": system},
                            {"role": "user", "content": prompt},
                        ],
                        model=self.model,
                    )
                    result = response.choices[0].message.content
                    if self.verbose:
                        print(f"Eviction LLM Result: {result}")
                    if result.lower() == "yes":
                        evictions.append(entity)
                        self.entities.remove(entity)
                        if self.verbose:
                            print(f"Entity {entity} is evicted.")
                    else:
                        if self.verbose:
                            print(f"Entity {entity} is not evicted.")
                except Exception as e:
                    error_message = f"An error occurred while checking if the entity should be evicted."
                    print(error_message)
                    if self.debug:
                        print(f"Error: {e}")
                        traceback.print_exc()
                    continue
        return evictions

In [ ]:
wayfarer = Wayfarer()

print("🚀 Wayfarer is ready to use. Continue to the next cell.")

In [ ]:
# Simple example using 'user records' as entities and 'age remover' as an event.

import json

wayfarer.add_entity({"name": "Alien Genius", "age": 12})
wayfarer.add_entity({"name": "Superman", "age": 9})
entites = wayfarer.get_entities()
print(f"Entites: {json.dumps(entites, indent=4)}")
wayfarer.add_event(
    {
        "id": "1",
        "name": "age remover",
        "rule": "Remove if Age > 10",
        "metadata": {"client_id": "1234"},
    }
)
wayfarer.add_entity({"object": "Rock", "color": "black"})
evictions = wayfarer.get_evictions()
print(f"Evictions: {json.dumps(evictions, indent=4)}")
entites_after_eviction = wayfarer.get_entities()
print(f"Entites after eviction: {json.dumps(entites_after_eviction, indent=4)}")

In [ ]:
# Advanced example using mythical creatures as entities and 'elemental purge' as an event.

import json

# Adding mythical entities to the system
wayfarer.add_entity({"name": "Phoenix", "age": 500, "element": "Fire"})
wayfarer.add_entity({"name": "Mermaid", "age": 200, "element": "Water"})
wayfarer.add_entity({"name": "Unicorn", "age": 50, "element": "Earth"})

# Fetching all entities
entities = wayfarer.get_entities()
print(f"Entities: {json.dumps(entities, indent=4)}")

# Adding a magical event to the system
wayfarer.add_event(
    {
        "id": "101",
        "name": "Elemental Purge",
        "rule": "Remove if Element is 'Fire'",
        "metadata": {"wizard_id": "Albus123"},
    }
)

# Adding more mythical creatures
wayfarer.add_entity({"name": "Dragon", "age": 800, "element": "Fire"})
wayfarer.add_entity({"name": "Fairy", "age": 150, "element": "Air"})

# Checking which entities were evicted based on the magical event rule
evictions = wayfarer.get_evictions()
print(f"Evictions: {json.dumps(evictions, indent=4)}")

# Fetching the list of remaining entities after applying the event
entities_after_eviction = wayfarer.get_entities()
print(f"Entities after eviction: {json.dumps(entities_after_eviction, indent=4)}")